## Elective Objectives / Ideas


#### Objectives:
1. Basics: RL
2. Python basics: numpy, pandas and matplotlib
3. Advanced: PyTorch, RL programming, TensorBoard
4. Understand OpenAI gym and create **custom env.** 
5. Put together a RL loop - non-functional/dummy
6. Show training curves in TensorBoard
7. **Propose ideas for a functional model**

#### Approach/Ideas:
1. Action: Is one of No-action, Maintenance-Regular, Maintenance-Special, Replace
2. Cost: Associate cost of taking these: Replace = 100, Special=40, Regular=20
3. After each maintenance: Increase RUL: Replace = x10, Special=x5, Regular=x2
4. Doing regular in downward trend of operation -> RUL increase only 10% regular
5. Reward: Most life, least cost so for example: R = RUL/(Cost+1) i.e. as RUL high and cost low -> reward MOST 
6. What one expects: Regular maintenance done in normal cycle
7. Optimal Policy = When to maintain with least cost

- Running TensorBoard: tensorboard --logdir D:\Rajesh\lab\\_elective_\tensorboard\.

In [21]:
import gym
import json

from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C, PPO, DQN

In [22]:
import random
import math
import json
import gym
from gym import spaces
import pandas as pd
import numpy as np

MAX_LIFE = 470

MAX_VIBRATION = 2
MAX_REWARD = 10
MAX_STEPS = 20000
RUL_WINDOW = 10
LAMBDA = 0.01
INITIAL_RUL = MAX_LIFE

NO_ACTION = 0
MAINTENANCE_REGULAR = 1
MAINTENANCE_SPECIAL = 2
MAINTENANCE_REPLACE = 3

a_cost = []
a_action_recommended = []
a_actions = []
a_action_text = []
a_rul = []
a_rewards = []
a_events = []
a_time = []

class BearingEnv(gym.Env):
    """A Bearing RUL estimating environment for OpenAI gym"""
    metadata = {'render.modes': ['human']}

    def __init__(self, df):        
        super(BearingEnv, self).__init__()

        self.df = df
        self.maintenance_cost = 0
        self.maintenance_events = 0
        self.rul = 0
        self.reward = 0
        self.current_action = NO_ACTION
        
        self.reward_range = (0, MAX_REWARD)

        high = np.array(
            [
                MAX_LIFE,
                MAX_VIBRATION,
                MAINTENANCE_REPLACE,
            ],
            dtype=np.float32,
        )
        
        self.action_space = spaces.Discrete(4)
        self.observation_space = spaces.Box(-high, high, dtype=np.float32)
        
        self.state = None
            
    def _next_observation(self):      
        frame = np.array([
            self.df.loc[self.current_step, 'RUL'] / MAX_LIFE,
            self.df.loc[self.current_step, 'vibration'] / MAX_VIBRATION,
            self.df.loc[self.current_step, 'temperature'],
        ])
        
        self.rul = self.df.loc[self.current_step: self.current_step, 'RUL'].values[0]
        
        # Append additional data and scale each value to between 0-1
        obs = frame.flatten()
        return obs

    def _take_action(self, action):

        if action == NO_ACTION: # Normal state
            # 1% reduction in life
            # self.rul *= 0.99
            # self.maintenance_cost += 0.1
            action_text = 'None'
            
        elif action == MAINTENANCE_REGULAR: 
            # 1% increase in life
            self.rul *= 1.01
            self.maintenance_cost += 1
            self.maintenance_events += 1
            action_text = 'Maintenance-Regular'

        elif action == MAINTENANCE_SPECIAL: 
            # 5% increase in life
            self.rul *= 1.05
            self.maintenance_cost += 4
            self.maintenance_events += 1
            action_text = 'Maintenance-Special' 
            
        elif action == MAINTENANCE_REPLACE: 
            # Normal state
            self.rul = MAX_LIFE
            self.maintenance_cost += 20
            self.maintenance_events += 1
            action_text = '* REPLACE *' 
            
        #a_actions.append(action)
        a_action_text.append(action_text)
        self.reward = self.rul / (self.maintenance_cost+LAMBDA)
        
        print('{0:<20} | RUL: {1:>8.2f} | Cost: {2:>8.2f} | Maintenance events: {3:>3d} | Reward: {4:>12.3f}'.
              format(action_text, self.rul, self.maintenance_cost, self.maintenance_events, self.reward))
                 
        self.state = (self.rul, self.maintenance_cost, action)

  
    def step(self, action):        
        # Execute one time step within the environment
        self._take_action(action)
        reward = self.reward
        
        if self.current_step >= (len(self.df.loc[:, 'RUL'].values)-1):
            done = True
        else:
            self.current_step += 1
            done = False
                
        if self.rul <= 10:
            done = True
        elif self.rul >= MAX_LIFE:
            done = True
        else:
            done = False

        obs = self._next_observation()
        
        a_time.append(self.current_step)
        a_rewards.append(self.reward)
        a_rul.append(self.rul)
        a_cost.append(self.maintenance_cost)
        a_actions.append(action)
        a_events.append(self.maintenance_events)
        
        # From database extract recommended action
        recommended_action = self.df.loc[self.current_step: self.current_step, 'ACTION_CODE'].values[0]
        a_action_recommended.append(recommended_action)
        
        return obs, reward, done, {}

    def reset(self):
        done = False
        self.state = np.random.uniform(low=-0.0, high=1.0, size=(3,))

        # Set the current step to a random point within the data frame
        self.current_step = 0
        # self.current_step = random.randint(0, len(self.df.loc[:, 'RUL'].values))

        self.rul = self.df.loc[self.current_step: self.current_step, 'RUL'].values[0]
        print('\n --- RESET. Starting RUL = ', self.rul)
        # Reset the state of the environment to an initial state
        self.maintenance_cost = 0
        self.maintenance_events = 0
        self.reward = 0
        self.current_action = NO_ACTION
        
        return np.array(self.state, dtype=np.float32)

    def render(self, mode='human', close=False):
        # Render the environment to the screen
        RUL = self.rul
        
        print('>> {0:<20} | RUL: {1:>8.2f} | Cost: {2:>8.2f} | Reward: {3:>12.3f}'.
              format(self.current_action, self.rul, self.maintenance_cost, self.reward))

In [23]:
# from stable_baselines3.common.env_checker import check_env
# env = BearingEnv(df)
# check_env(env)

In [24]:
df = pd.read_csv('data\data.csv')

In [25]:
EPISODES = 4000
RUN_TEST = 20

In [26]:
# The algorithms require a vectorized environment to run
env = DummyVecEnv([lambda: BearingEnv(df)])

results_file = 'Test_run_A2C.csv'

model = A2C('MlpPolicy', env, verbose=1, tensorboard_log="./tensorboard/")
#model = DQN('MlpPolicy', env, verbose=1, tensorboard_log="./tensorboard/")
#model = PPO('MlpPolicy', env, verbose=1, tensorboard_log="./tensorboard/")

model.learn(total_timesteps=EPISODES)

Using cpu device

 --- RESET. Starting RUL =  461.331
Logging to ./tensorboard/A2C_62
None                 | RUL:   461.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    46133.100
Maintenance-Regular  | RUL:   460.89 | Cost:     1.00 | Maintenance events:   1 | Reward:      456.331
Maintenance-Regular  | RUL:   455.84 | Cost:     2.00 | Maintenance events:   2 | Reward:      226.788
Maintenance-Regular  | RUL:   450.79 | Cost:     3.00 | Maintenance events:   3 | Reward:      149.766
Maintenance-Special  | RUL:   463.40 | Cost:     7.00 | Maintenance events:   4 | Reward:       66.105
Maintenance-Regular  | RUL:   440.69 | Cost:     8.00 | Maintenance events:   5 | Reward:       55.018
Maintenance-Special  | RUL:   452.90 | Cost:    12.00 | Maintenance events:   6 | Reward:       37.710
Maintenance-Regular  | RUL:   430.59 | Cost:    13.00 | Maintenance events:   7 | Reward:       33.097
None                 | RUL:   421.33 | Cost:    13.00 | Maintenance events:   7 | Reward: 

None                 | RUL:   396.33 | Cost:    17.00 | Maintenance events:   8 | Reward:       23.300
Maintenance-Special  | RUL:   410.90 | Cost:    21.00 | Maintenance events:   9 | Reward:       19.557
Maintenance-Regular  | RUL:   390.19 | Cost:    22.00 | Maintenance events:  10 | Reward:       17.728
Maintenance-Regular  | RUL:   385.14 | Cost:    23.00 | Maintenance events:  11 | Reward:       16.738
Maintenance-Special  | RUL:   395.15 | Cost:    27.00 | Maintenance events:  12 | Reward:       14.630
Maintenance-Regular  | RUL:   375.04 | Cost:    28.00 | Maintenance events:  13 | Reward:       13.390
Maintenance-Regular  | RUL:   369.99 | Cost:    29.00 | Maintenance events:  14 | Reward:       12.754
Maintenance-Special  | RUL:   379.40 | Cost:    33.00 | Maintenance events:  15 | Reward:       11.493
None                 | RUL:   356.33 | Cost:    33.00 | Maintenance events:  15 | Reward:       10.795
Maintenance-Special  | RUL:   368.90 | Cost:    37.00 | Maintenance event

None                 | RUL:   256.33 | Cost:    18.00 | Maintenance events:   9 | Reward:       14.233
None                 | RUL:   251.33 | Cost:    18.00 | Maintenance events:   9 | Reward:       13.955
None                 | RUL:   246.33 | Cost:    18.00 | Maintenance events:   9 | Reward:       13.677
None                 | RUL:   241.33 | Cost:    18.00 | Maintenance events:   9 | Reward:       13.400
None                 | RUL:   236.33 | Cost:    18.00 | Maintenance events:   9 | Reward:       13.122
None                 | RUL:   231.33 | Cost:    18.00 | Maintenance events:   9 | Reward:       12.845
None                 | RUL:   226.33 | Cost:    18.00 | Maintenance events:   9 | Reward:       12.567
None                 | RUL:   222.33 | Cost:    18.00 | Maintenance events:   9 | Reward:       12.345
None                 | RUL:   221.83 | Cost:    18.00 | Maintenance events:   9 | Reward:       12.317
None                 | RUL:   221.61 | Cost:    18.00 | Maintenance event

None                 | RUL:   346.33 | Cost:    18.00 | Maintenance events:   6 | Reward:       19.230
None                 | RUL:   341.33 | Cost:    18.00 | Maintenance events:   6 | Reward:       18.952
None                 | RUL:   336.33 | Cost:    18.00 | Maintenance events:   6 | Reward:       18.675
None                 | RUL:   331.33 | Cost:    18.00 | Maintenance events:   6 | Reward:       18.397
None                 | RUL:   326.33 | Cost:    18.00 | Maintenance events:   6 | Reward:       18.119
None                 | RUL:   321.33 | Cost:    18.00 | Maintenance events:   6 | Reward:       17.842
None                 | RUL:   316.33 | Cost:    18.00 | Maintenance events:   6 | Reward:       17.564
None                 | RUL:   311.33 | Cost:    18.00 | Maintenance events:   6 | Reward:       17.287
Maintenance-Regular  | RUL:   309.39 | Cost:    19.00 | Maintenance events:   7 | Reward:       16.275
Maintenance-Special  | RUL:   316.40 | Cost:    23.00 | Maintenance event

None                 | RUL:   207.67 | Cost:    53.00 | Maintenance events:  17 | Reward:        3.918
None                 | RUL:   202.81 | Cost:    53.00 | Maintenance events:  17 | Reward:        3.826
Maintenance-Special  | RUL:   208.45 | Cost:    57.00 | Maintenance events:  18 | Reward:        3.656
None                 | RUL:   194.60 | Cost:    57.00 | Maintenance events:  18 | Reward:        3.413
None                 | RUL:   190.91 | Cost:    57.00 | Maintenance events:  18 | Reward:        3.349
None                 | RUL:   187.38 | Cost:    57.00 | Maintenance events:  18 | Reward:        3.287
None                 | RUL:   183.94 | Cost:    57.00 | Maintenance events:  18 | Reward:        3.226
None                 | RUL:   180.53 | Cost:    57.00 | Maintenance events:  18 | Reward:        3.167
None                 | RUL:   177.11 | Cost:    57.00 | Maintenance events:  18 | Reward:        3.107
None                 | RUL:   173.63 | Cost:    57.00 | Maintenance event

...

None                 | RUL:    45.00 | Cost:     8.00 | Maintenance events:   2 | Reward:        5.618
None                 | RUL:    31.00 | Cost:     8.00 | Maintenance events:   2 | Reward:        3.870
None                 | RUL:    16.00 | Cost:     8.00 | Maintenance events:   2 | Reward:        1.998
None                 | RUL:     0.00 | Cost:     8.00 | Maintenance events:   2 | Reward:        0.000

 --- RESET. Starting RUL =  461.331
None                 | RUL:   461.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    46133.100
None                 | RUL:   456.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    45633.100
None                 | RUL:   451.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    45133.100
None                 | RUL:   446.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    44633.100
None                 | RUL:   441.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    44133.100
None                 | RUL:   436.33

None                 | RUL:    91.00 | Cost:     4.00 | Maintenance events:   1 | Reward:       22.693
None                 | RUL:    81.00 | Cost:     4.00 | Maintenance events:   1 | Reward:       20.200
None                 | RUL:    70.00 | Cost:     4.00 | Maintenance events:   1 | Reward:       17.456
None                 | RUL:    58.00 | Cost:     4.00 | Maintenance events:   1 | Reward:       14.464
None                 | RUL:    45.00 | Cost:     4.00 | Maintenance events:   1 | Reward:       11.222
None                 | RUL:    31.00 | Cost:     4.00 | Maintenance events:   1 | Reward:        7.731
None                 | RUL:    16.00 | Cost:     4.00 | Maintenance events:   1 | Reward:        3.990
None                 | RUL:     0.00 | Cost:     4.00 | Maintenance events:   1 | Reward:        0.000

 --- RESET. Starting RUL =  461.331
None                 | RUL:   461.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    46133.100
None                 | RUL:   456.33

None                 | RUL:    58.00 | Cost:     4.00 | Maintenance events:   1 | Reward:       14.464
None                 | RUL:    45.00 | Cost:     4.00 | Maintenance events:   1 | Reward:       11.222
None                 | RUL:    31.00 | Cost:     4.00 | Maintenance events:   1 | Reward:        7.731
None                 | RUL:    16.00 | Cost:     4.00 | Maintenance events:   1 | Reward:        3.990
None                 | RUL:     0.00 | Cost:     4.00 | Maintenance events:   1 | Reward:        0.000

 --- RESET. Starting RUL =  461.331
None                 | RUL:   461.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    46133.100
None                 | RUL:   456.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    45633.100
None                 | RUL:   451.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    45133.100
None                 | RUL:   446.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    44633.100
None                 | RUL:   441.33

None                 | RUL:   441.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    44133.100
None                 | RUL:   436.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    43633.100
None                 | RUL:   431.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    43133.100
None                 | RUL:   426.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    42633.100
None                 | RUL:   421.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    42133.100
None                 | RUL:   416.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    41633.100
None                 | RUL:   411.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    41133.100
None                 | RUL:   406.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    40633.100
None                 | RUL:   401.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    40133.100
None                 | RUL:   396.33 | Cost:     0.00 | Maintenance event

None                 | RUL:   391.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    39133.100
None                 | RUL:   386.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    38633.100
None                 | RUL:   381.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    38133.100
None                 | RUL:   376.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    37633.100
None                 | RUL:   371.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    37133.100
None                 | RUL:   366.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    36633.100
None                 | RUL:   361.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    36133.100
None                 | RUL:   356.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    35633.100
None                 | RUL:   351.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    35133.100
None                 | RUL:   346.33 | Cost:     0.00 | Maintenance event

In [27]:
df_results = pd.DataFrame({'Time-step':a_time[:], 'Recommended_Action':a_action_recommended[:], 'Action':a_action_text[:], 
                           'Action_Code':a_actions[:], 'Rewards':a_rewards[:], 'RUL':a_rul[:], 
                           'MaintenanceEvents':a_events[:], 'Cost':a_cost[:]})
df_results.head()

,Time-step,Recommended_Action,Action,Action_Code,Rewards,RUL,MaintenanceEvents,Cost
0,1,0,None,0,46133.100000,456.331,0,0
1,2,0,Maintenance-Regular,1,456.331000,451.331,1,1
2,3,0,Maintenance-Regular,1,226.788214,446.331,2,2
3,4,0,Maintenance-Regular,1,149.765551,441.331,3,3
4,5,0,Maintenance-Special,2,66.105214,436.331,4,7


In [28]:
df_results.Rewards = df_results.Rewards/df_results.Rewards.max()
df_results.RUL = df_results.RUL/df_results.RUL.max()
df_results.Cost = df_results.Cost/df_results.Cost.max()

In [29]:
df_results.to_csv(results_file)

In [30]:
obs = env.reset()
for i in range(20):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()


 --- RESET. Starting RUL =  461.331
None                 | RUL:   461.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    46133.100
>> 0                    | RUL:   456.33 | Cost:     0.00 | Reward:    46133.100
None                 | RUL:   456.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    45633.100
>> 0                    | RUL:   451.33 | Cost:     0.00 | Reward:    45633.100
None                 | RUL:   451.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    45133.100
>> 0                    | RUL:   446.33 | Cost:     0.00 | Reward:    45133.100
None                 | RUL:   446.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    44633.100
>> 0                    | RUL:   441.33 | Cost:     0.00 | Reward:    44633.100
None                 | RUL:   441.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    44133.100
>> 0                    | RUL:   436.33 | Cost:     0.00 | Reward:    44133.100
None                 | RUL:   436.33 | Cost:    